In [1]:
import kagglehub
path = kagglehub.dataset_download("hassanmojab/xview-dataset")

print("✅ Downloaded to:", path)

100%|██████████| 19.4G/19.4G [03:16<00:00, 106MB/s]

Extracting files...


✅ Downloaded to: /root/.cache/kagglehub/datasets/hassanmojab/xview-dataset/versions/1


In [2]:
!pip install -q ultralytics geopandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 117.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 104.6 MB/s eta 0:00:00


In [3]:
import os

path = "/root/.cache/kagglehub/datasets/hassanmojab/xview-dataset/versions/1"
print("📂 Contents of downloaded dataset:")
print(os.listdir(path))

📂 Contents of downloaded dataset:
['__notebook_source__.ipynb', 'train_images', 'val_images', 'train_labels']


In [2]:
import os
import json
import shutil
import yaml
from collections import Counter
from PIL import Image
from ultralytics import YOLO

In [5]:
# ===============================
# 📂 DATASET PATHS
# ===============================
base_path = "/root/.cache/kagglehub/datasets/hassanmojab/xview-dataset/versions/1"
image_source = os.path.join(base_path, "train_images", "train_images")
geojson_path = os.path.join(base_path, "train_labels", "xView_train.geojson")

In [6]:
# ===============================
# 📁 OUTPUT STRUCTURE
# ===============================
os.makedirs("images/train", exist_ok=True)
os.makedirs("images/val", exist_ok=True)
os.makedirs("labels/train", exist_ok=True)
os.makedirs("labels/val", exist_ok=True)

In [7]:
# === CLASS MAPPING ===
xview_classes = {
    0: "Fixed-wing Aircraft", 1: "Small Aircraft", 2: "Cargo Plane", 3: "Helicopter",
    4: "Passenger Vehicle", 5: "Small Car", 6: "Bus", 7: "Pickup Truck", 8: "Utility Truck",
    9: "Truck", 10: "Cargo Truck", 11: "Truck w/Box", 12: "Truck Tractor", 13: "Trailer",
    14: "Truck w/Flatbed", 15: "Truck w/Liquid", 16: "Crane Truck", 17: "Railway Vehicle",
    18: "Passenger Car", 19: "Cargo Car", 20: "Flat Car", 21: "Tank car", 22: "Locomotive",
    23: "Container Ship", 24: "Liner Ship", 25: "Fishing Vessel", 26: "Oil Tanker",
    27: "Engineering Vehicle", 28: "Tower crane", 29: "Container Crane", 30: "Reach Stacker",
    31: "Straddle Carrier", 32: "Mobile Crane", 33: "Dump Truck", 34: "Haul Truck",
    35: "Scraper/Tractor", 36: "Front Loader/Bulldozer", 37: "Excavator", 38: "Cement Mixer",
    39: "Ground Grader", 40: "Hut/Tent", 41: "Shed", 42: "Building", 43: "Aircraft Hangar",
    44: "Damaged Building", 45: "Facility", 46: "Construction Site", 47: "Vehicle Lot",
    48: "Helipad", 49: "Storage Tank", 50: "Shipping Container Lot", 51: "Shipping Container",
    52: "Pylon", 53: "Tower", 54: "Wind Turbine", 55: "Tower-like Structure", 56: "Container/Truck",
    57: "Oil Rig", 58: "Pier", 59: "Dock"
}
sorted_ids = sorted(xview_classes.keys())
typeid_to_yolo = {tid: i for i, tid in enumerate(sorted_ids)}

In [8]:
# === PARSE GEOJSON ===
with open(geojson_path) as f:
    data = json.load(f)

annotations = {}
for feat in data["features"]:
    props = feat["properties"]
    image_id = props.get("image_id")
    bbox = props.get("bounds_imcoords")
    cls = props.get("type_id")

    if not image_id or not bbox or cls not in typeid_to_yolo:
        continue

    try:
        x1, y1, x2, y2 = map(float, bbox.split(","))
    except:
        continue

    annotations.setdefault(image_id, []).append((typeid_to_yolo[cls], x1, y1, x2, y2))


In [9]:
# === SPLIT 80/20 ===
import random
image_ids = list(annotations.keys())
random.shuffle(image_ids)
split = int(len(image_ids) * 0.8)
train_ids, val_ids = image_ids[:split], image_ids[split:]

In [10]:
# === COPY IMAGES + CONVERT LABELS ===
def process_set(image_ids, img_dst, lbl_dst):
    count = 0
    for img_name in image_ids:
        src = os.path.join(image_source, img_name)
        dst = os.path.join(img_dst, img_name)
        if not os.path.exists(src):
            continue
        try:
            with Image.open(src) as im:
                w, h = im.size
        except:
            continue
        shutil.copy(src, dst)

        label_file = os.path.splitext(img_name)[0] + ".txt"
        label_path = os.path.join(lbl_dst, label_file)

        with open(label_path, "w") as f:
            for cls, x1, y1, x2, y2 in annotations[img_name]:
                xc = ((x1 + x2) / 2) / w
                yc = ((y1 + y2) / 2) / h
                bw = (x2 - x1) / w
                bh = (y2 - y1) / h
                f.write(f"{cls} {xc:.6f} {yc:.6f} {bw:.6f} {bh:.6f}\n")
        count += 1
    return count

train_count = process_set(train_ids, "images/train", "labels/train")
val_count   = process_set(val_ids,   "images/val",   "labels/val")

print(f"✅ {train_count} training images and {val_count} validation images processed")

# === WRITE data.yaml ===
data_yaml = {
    "train": "images/train",
    "val": "images/val",
    "nc": len(xview_classes),
    "names": [xview_classes[i] for i in sorted_ids]
}
with open("data.yaml", "w") as f:
    yaml.dump(data_yaml, f)

print("✅ data.yaml written — ready to train!")


✅ 614 training images and 154 validation images processed
✅ data.yaml written — ready to train!


In [3]:
model = YOLO("yolov8m.pt")  # Use 'n', 's', 'l', or 'x' as needed

model.train(
    data="data.yaml",       # Path to your dataset config
    epochs=30,              # Reduced from 60 to fit Colab session (~1–2 hrs max)
    imgsz=1024,              # Slightly smaller to fit memory; 1024 might OOM at higher batch
    batch=2,                # Optimal for T4 (~15–16GB); reduce to 2 if OOM
    patience=6,             # Early stopping patience
    workers=2,              # Number of dataloader workers
    name="satellite_model", # Run name
    augment=True,           # Enable strong data augmentation
    pretrained=True,        # Leverage pretrained backbone
    verbose=True,           # Detailed logs
    lr0=0.0015,             # Slightly tuned initial learning rate
    warmup_epochs=2,        # Warmup to stabilize early training
    cos_lr=True             # Cosine LR scheduler for better convergence
)


Ultralytics 8.3.145 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=2, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=1024, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0015, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=satellite_model2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=6, perspective=0.0, plots=True, pose=12.0, pretrained=True, pro

train: Scanning /content/labels/train.cache... 614 images, 0 backgrounds, 176 corrupt: 100%|██████████| 614/614 [00:00<?, ?it/s]

train: /content/images/train/1036.tif: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0003      1.0003]
train: /content/images/train/1037.tif: ignoring corrupt image/label: negative label values [  -0.001095   -0.000156   -0.000782]
train: /content/images/train/1046.tif: ignoring corrupt image/label: negative label values [  -0.001095]
train: /content/images/train/1050.tif: ignoring corrupt image/label: negative label values [    -0.0014]
train: /content/images/train/1051.tif: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0012]
train: /content/images/train/107.tif: ignoring corrupt image/label: negative label values [  -0.065401]
train: /content/images/train/1076.tif: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0003]
train: /content/images/train/1080.tif: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0055       1.007      1.0052]
train: /conte

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.1±0.0 ms, read: 110.9±43.2 MB/s, size: 28254.6 KB)


val: Scanning /content/labels/val.cache... 154 images, 0 backgrounds, 40 corrupt: 100%|██████████| 154/154 [00:00<?, ?it/s]

...

                   all        114      35661      0.323     0.0658     0.0624     0.0283


EarlyStopping: Training stopped early as no improvement observed in last 6 epochs. Best results observed at epoch 9, best model saved as best.pt.
To update EarlyStopping(patience=6) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.

15 epochs completed in 0.496 hours.
Optimizer stripped from runs/detect/satellite_model2/weights/last.pt, 52.2MB
Optimizer stripped from runs/detect/satellite_model2/weights/best.pt, 52.2MB

Validating runs/detect/satellite_model2/weights/best.pt...
Ultralytics 8.3.145 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 92 layers, 25,874,500 parameters, 0 gradients, 78.9 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 29/29 [00:35<00:00,  1.21s/it]


                   all        114      35661      0.283     0.0626     0.0583     0.0282
           Truck w/Box          4          7          0          0          0          0
         Truck Tractor          7         16     0.0337      0.188     0.0266     0.0138
               Trailer         19         93      0.442      0.892      0.869      0.438
        Truck w/Liquid          3         10          0          0          0          0
       Railway Vehicle         31        383          1          0          0          0
         Passenger Car        105      29316      0.264     0.0569     0.0894     0.0228
             Cargo Car         63       1101      0.365     0.0418       0.08     0.0328
              Flat Car         30        174          1          0          0          0
              Tank car         51        462          0          0    0.00212   0.000318
        Container Ship         81       1612     0.0717     0.0199     0.0184    0.00659
            Liner Shi

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([11, 12, 13, 15, 17, 18, 19, 20, 21, 23, 24, 25, 26, 27, 28, 29, 32, 34, 35, 36, 37, 38, 40, 41, 42, 44, 47, 49, 51, 52, 53, 54, 55, 56, 57, 59])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7e9851aad8d0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.0380

In [4]:
from ultralytics import YOLO

# Load your trained model (replace with your model path if different)
model = YOLO("/content/runs/detect/satellite_model2/weights/best.pt")

# Run validation
metrics = model.val(
    data="data.yaml",   # Path to your data YAML file
    imgsz=1024,         # Match training image size
    batch=2,            # Same or smaller batch to avoid OOM
    verbose=True
)


Ultralytics 8.3.145 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 92 layers, 25,874,500 parameters, 0 gradients, 78.9 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1440.1±876.4 MB/s, size: 29037.5 KB)


val: Scanning /content/labels/val.cache... 154 images, 0 backgrounds, 40 corrupt: 100%|██████████| 154/154 [00:00<?, ?it/s]

val: /content/images/val/100.tif: ignoring corrupt image/label: negative label values [  -0.001924]
val: /content/images/val/1063.tif: ignoring corrupt image/label: negative label values [  -0.003622]
val: /content/images/val/1094.tif: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0011      1.0014]
val: /content/images/val/1104.tif: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0114      1.0014]
val: /content/images/val/1126.tif: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0007]
val: /content/images/val/1133.tif: ignoring corrupt image/label: non-normalized or out of bounds coordinates [      1.001]
val: /content/images/val/1141.tif: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0183      1.0014]
val: /content/images/val/1142.tif: ignoring corrupt image/label: negative label values [  -0.000756]
val: /content/images/val/1192.tif: ignoring cor


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 57/57 [00:22<00:00,  2.50it/s]


                   all        114      35661      0.309     0.0631     0.0678     0.0301
           Truck w/Box          4          7          0          0    0.00521   0.000521
         Truck Tractor          7         16     0.0423      0.188      0.034     0.0188
               Trailer         19         93      0.612      0.882      0.872      0.444
        Truck w/Liquid          3         10          0          0          0          0
       Railway Vehicle         31        383          1          0          0          0
         Passenger Car        105      29316      0.322       0.05     0.0994     0.0261
             Cargo Car         63       1101      0.521     0.0493     0.0903     0.0364
              Flat Car         30        174          1          0          0          0
              Tank car         51        462          0          0     0.0104    0.00225
        Container Ship         81       1612     0.0848      0.018     0.0208    0.00708
            Liner Shi

In [ ]:
import os
import matplotlib.pyplot as plt
import cv2

# Change this if the val run directory name differs
val_dir = "runs/val/predict"
images = [os.path.join(val_dir, img) for img in os.listdir(val_dir) if img.endswith((".jpg", ".png"))]

# Display a few sample predictions
for i, img_path in enumerate(images[:5]):  # Show first 5 images
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(10, 10))
    plt.imshow(img)
    plt.title(f"Prediction {i+1}")
    plt.axis('off')
    plt.show()


In [5]:
model = YOLO('/content/runs/detect/satellite_model2/weights/best.pt')


In [ ]:
# Path to your input image
image_path = "/content/3.png"

# Run inference
results = model(image_path)

In [ ]:
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt

# 1. Load your trained model
model = YOLO('/content/runs/detect/satellite_model2/weights/best.pt')

# 2. Run inference on the image
results = model("/content/stitched_satellite_image.png", imgsz=1024, conf=0.25, save=True)  # Adjust conf threshold as needed

# 3. Display the result
result_img = results[0].plot()  # Returns a numpy image with bounding boxes

plt.figure(figsize=(12, 12))
plt.imshow(result_img)
plt.axis("off")
plt.title("Detected Objects")
plt.show()

In [10]:
!pip install -q opencv-python ultralytics

In [11]:
def tile_and_predict(image_path, tile_size=1024, stride=1024, conf_thresh=0.3):
    image = cv2.imread(image_path)
    h, w, _ = image.shape
    results_all = []

    for y in range(0, h, stride):
        for x in range(0, w, stride):
            tile = image[y:y+tile_size, x:x+tile_size]
            if tile.shape[0] < tile_size or tile.shape[1] < tile_size:
                continue

            result = model(tile, conf=conf_thresh)[0]

            if result.boxes is not None:
                for box in result.boxes:
                    xyxy = box.xyxy[0].clone()  # clone fixes RuntimeError
                    x1, y1, x2, y2 = xyxy
                    x1 += x
                    x2 += x
                    y1 += y
                    y2 += y
                    results_all.append((int(box.cls[0]), float(box.conf[0]), x1, y1, x2, y2))

    return results_all


In [12]:
def get_color(cls_id):
    np.random.seed(cls_id)
    return tuple(int(x) for x in np.random.randint(0, 255, 3))

def draw_boxes(image_path, detections, class_names, output_path="result.jpg"):
    image = cv2.imread(image_path)
    for cls_id, conf, x1, y1, x2, y2 in detections:
        label = f"{class_names[cls_id]} {conf:.2f}"
        color = get_color(cls_id)
        cv2.rectangle(image, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)
        cv2.putText(image, label, (int(x1), int(y1) - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
    cv2.imwrite(output_path, image)
    print(f"✅ Output saved as {output_path}")

In [13]:
detections = tile_and_predict("/content/stitched_satellite_image.png")


0: 1024x1024 (no detections), 77.5ms
Speed: 6.8ms preprocess, 77.5ms inference, 0.9ms postprocess per image at shape (1, 3, 1024, 1024)

0: 1024x1024 (no detections), 68.2ms
Speed: 4.9ms preprocess, 68.2ms inference, 0.9ms postprocess per image at shape (1, 3, 1024, 1024)

0: 1024x1024 (no detections), 55.1ms
Speed: 4.9ms preprocess, 55.1ms inference, 0.8ms postprocess per image at shape (1, 3, 1024, 1024)

0: 1024x1024 (no detections), 54.9ms
Speed: 4.7ms preprocess, 54.9ms inference, 0.8ms postprocess per image at shape (1, 3, 1024, 1024)

0: 1024x1024 (no detections), 53.8ms
Speed: 5.3ms preprocess, 53.8ms inference, 0.7ms postprocess per image at shape (1, 3, 1024, 1024)

0: 1024x1024 (no detections), 48.8ms
Speed: 4.9ms preprocess, 48.8ms inference, 3.3ms postprocess per image at shape (1, 3, 1024, 1024)

0: 1024x1024 (no detections), 46.7ms
Speed: 7.3ms preprocess, 46.7ms inference, 3.4ms postprocess per image at shape (1, 3, 1024, 1024)

0: 1024x1024 1 Trailer, 44.4ms
Speed: 11

In [15]:
import numpy as np
# Now call draw_boxes with the generated detections
draw_boxes("/content/stitched_satellite_image.png", detections, model.names)

✅ Output saved as result.jpg


In [20]:
import os
import cv2
from ultralytics import YOLO

# Load the model
model = YOLO('/content/runs/detect/satellite_model2/weights/best.pt')


# Load your large image
img = cv2.imread("/content/stitched_satellite_image.png")
H, W, _ = img.shape

tile_size = 1024
stride = 1024  # You can use <1024 for overlapping tiles
os.makedirs("detected_tiles", exist_ok=True)

# Slide window across image
for y in range(0, H, stride):
    for x in range(0, W, stride):
        tile = img[y:y + tile_size, x:x + tile_size]

        if tile.shape[0] != tile_size or tile.shape[1] != tile_size:
            continue  # Skip edge tiles if smaller than expected

        # Save temp tile
        tile_path = "temp_tile.png"
        cv2.imwrite(tile_path, tile)

        # Run YOLO on the tile
        results = model(tile_path, imgsz=1024, conf=0.25, verbose=False)

        # Check if objects were detected
        if len(results[0].boxes) > 0:
            save_path = f"detected_tiles/tile_{x}_{y}.png"
            cv2.imwrite(save_path, tile)
            print(f"✅ Detected objects in tile at ({x}, {y}) → saved.")
        else:
            print(f"⛔ No detection in tile at ({x}, {y}) → skipped.")


⛔ No detection in tile at (0, 0) → skipped.
⛔ No detection in tile at (1024, 0) → skipped.
⛔ No detection in tile at (2048, 0) → skipped.
✅ Detected objects in tile at (3072, 0) → saved.
⛔ No detection in tile at (4096, 0) → skipped.
⛔ No detection in tile at (5120, 0) → skipped.
⛔ No detection in tile at (6144, 0) → skipped.
✅ Detected objects in tile at (7168, 0) → saved.
✅ Detected objects in tile at (8192, 0) → saved.
✅ Detected objects in tile at (9216, 0) → saved.
⛔ No detection in tile at (0, 1024) → skipped.
⛔ No detection in tile at (1024, 1024) → skipped.
✅ Detected objects in tile at (2048, 1024) → saved.
⛔ No detection in tile at (3072, 1024) → skipped.
⛔ No detection in tile at (4096, 1024) → skipped.
⛔ No detection in tile at (5120, 1024) → skipped.
⛔ No detection in tile at (6144, 1024) → skipped.
✅ Detected objects in tile at (7168, 1024) → saved.
⛔ No detection in tile at (8192, 1024) → skipped.
⛔ No detection in tile at (9216, 1024) → skipped.
⛔ No detection in tile a

In [21]:
import shutil

# Replace 'folder_name' with the actual folder path you want to delete
folder_path = '/content/detected_tiles'

# Delete the folder
shutil.rmtree(folder_path)

print(f"✅ Deleted folder: {folder_path}")


✅ Deleted folder: /content/detected_tiles


In [25]:
import os
import cv2
from ultralytics import YOLO

# Load YOLOv8 model
# Load the model
model = YOLO('/content/runs/detect/satellite_model2/weights/best.pt')


# Load your large image
img = cv2.imread("/content/stitched_satellite_image.png")
H, W, _ = img.shape

# Parameters
tile_size = 1024
stride = 1024
os.makedirs("detected_tiles3", exist_ok=True)

# Class names
class_names = model.names  # {0: 'building', 1: 'vehicle', ...}

# Tiling and detection
for y in range(0, H, stride):
    for x in range(0, W, stride):
        tile = img[y:y + tile_size, x:x + tile_size]

        if tile.shape[0] != tile_size or tile.shape[1] != tile_size:
            continue

        # Save temp tile
        tile_path = "temp_tile.png"
        cv2.imwrite(tile_path, tile)

        # Inference
        results = model(tile_path, imgsz=1024, conf=0.25, verbose=False)

        # Check for detections
        if len(results[0].boxes) > 0:
            # Draw boxes and labels on the tile
            for box in results[0].boxes:
                # Get box coords
                x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
                cls_id = int(box.cls[0])
                conf = float(box.conf[0])

                label = f"{class_names[cls_id]} {conf:.2f}"

                # Draw rectangle and label
                cv2.rectangle(tile, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(tile, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX,
                            0.5, (255, 255, 255), 2, lineType=cv2.LINE_AA)

            # Save the annotated tile
            save_path = f"detected_tiles/tile_{x}_{y}.png"
            cv2.imwrite(save_path, tile)
            print(f"✅ Detected & saved annotated tile ({x}, {y})")
        else:
            print(f"⛔ No detection in tile ({x}, {y})")


⛔ No detection in tile (0, 0)
⛔ No detection in tile (1024, 0)
⛔ No detection in tile (2048, 0)
✅ Detected & saved annotated tile (3072, 0)
⛔ No detection in tile (4096, 0)
⛔ No detection in tile (5120, 0)
⛔ No detection in tile (6144, 0)
✅ Detected & saved annotated tile (7168, 0)
✅ Detected & saved annotated tile (8192, 0)
✅ Detected & saved annotated tile (9216, 0)
⛔ No detection in tile (0, 1024)
⛔ No detection in tile (1024, 1024)
✅ Detected & saved annotated tile (2048, 1024)
⛔ No detection in tile (3072, 1024)
⛔ No detection in tile (4096, 1024)
⛔ No detection in tile (5120, 1024)
⛔ No detection in tile (6144, 1024)
✅ Detected & saved annotated tile (7168, 1024)
⛔ No detection in tile (8192, 1024)
⛔ No detection in tile (9216, 1024)
⛔ No detection in tile (0, 2048)
✅ Detected & saved annotated tile (1024, 2048)
⛔ No detection in tile (2048, 2048)
✅ Detected & saved annotated tile (3072, 2048)
⛔ No detection in tile (4096, 2048)
⛔ No detection in tile (5120, 2048)
⛔ No detection 

In [26]:
!zip -r detected_tiles.zip detected_tiles


  adding: detected_tiles/ (stored 0%)
  adding: detected_tiles/tile_4096_4096.png (deflated 8%)
  adding: detected_tiles/tile_9216_6144.png (deflated 6%)
  adding: detected_tiles/tile_4096_6144.png (deflated 13%)
  adding: detected_tiles/tile_2048_1024.png (deflated 7%)
  adding: detected_tiles/tile_5120_4096.png (deflated 2%)
  adding: detected_tiles/tile_7168_0.png (deflated 5%)
  adding: detected_tiles/tile_3072_0.png (deflated 9%)
  adding: detected_tiles/tile_9216_9216.png (deflated 4%)
  adding: detected_tiles/tile_9216_3072.png (deflated 4%)
  adding: detected_tiles/tile_7168_1024.png (deflated 5%)
  adding: detected_tiles/tile_4096_5120.png (deflated 12%)
  adding: detected_tiles/tile_7168_9216.png (deflated 11%)
  adding: detected_tiles/tile_4096_3072.png (deflated 6%)
  adding: detected_tiles/tile_8192_0.png (deflated 4%)
  adding: detected_tiles/tile_3072_3072.png (deflated 14%)
  adding: detected_tiles/tile_2048_5120.png (deflated 15%)
  adding: detected_tiles/tile_9216_0.p

In [27]:
!zip -r detected_tiles2.zip detected_tiles2


  adding: detected_tiles2/ (stored 0%)


In [28]:
!zip -r detected_tiles3.zip detected_tiles3


  adding: detected_tiles3/ (stored 0%)
